# J5 — Itérateurs & Générateurs

## Objectifs
- Comprendre `iter()` / `next()` et le protocole itérateur (`__iter__`, `__next__`).
- Écrire des générateurs (`yield`) et des générateurs infinis.
- Pratiquer la lazy evaluation (lecture fichiers) et les pipelines.
- Renforcer la manipulation d'itérables en vue du module Flask.

## Plan (8 exos)
1) Itérateur de base (iter/next)
2) Itérateur personnalisé — pairs
3) Générateur — Fibonacci (n premiers)
4) Générateur infini — count_from
5) Générateur lazy — read_lines(path)
6) Pipeline de générateurs — numbers → squares → even_filter
7) Générateur — chunked(iterable, size)
8) Générateur — unique(iterable)

> Règle : pas de solution ici. Utilisez les `TODO` + petits tests (commentés).


## Exo 1 — Itérateur de base
- Créez une liste.
- Obtenez un itérateur avec `iter(...)`.
- Avancez avec `next(...)` plusieurs fois.
- Gérez proprement l'arrêt en attrapant `StopIteration` (juste pour l'afficher).


In [1]:
# TODO: créer une liste puis itérer avec iter/next
items = [1, 2, 3, 4]

it = iter(items)

# TODO: next(it) plusieurs fois
print(next(it))
print(next(it))
print(next(it))
print(next(it))

# TODO: démontrer StopIteration
try:
    print(next(it))
except StopIteration:
    print("Fin de l'itérateur")


1
2
3
4
Fin de l'itérateur


## Exo 2 — Itérateur personnalisé : EvenNumbers(n)
- Implémentez une classe itérable qui renvoie `0, 2, 4, ..., n` (inclus si pair).
- Méthodes à définir : `__iter__(self) -> self` et `__next__(self)`.
- Levez `StopIteration` quand terminé.


In [2]:
# TODO: classe EvenNumbers
class EvenNumbers:
    def __init__(self, n: int):
        # TODO: stocker n et un curseur interne (ex: cur=0)
        self.n = int(n)
        self.cur = 0

    def __iter__(self):
        # TODO: renvoyer self
        return self

    def __next__(self) -> int:
        # TODO:
        # - si cur > n -> StopIteration
        if self.cur > self.n:
            raise StopIteration

        val = self.cur
        self.cur += 2
        return val



# tests (dé commentez)
print(list(EvenNumbers(10)))   # attendu: [0, 2, 4, 6, 8, 10]


[0, 2, 4, 6, 8, 10]


## Exo 3 — Générateur `fibonacci(n)`
- Écrire `fibonacci(n)` qui `yield` les `n` premiers termes : 0, 1, 1, 2, 3, 5, ...
- Utiliser une boucle et le couple `(a, b)`.


In [3]:
# TODO: générateur fibonacci(n)
def fibonacci(n: int):
    # TODO: yield n valeurs
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

# tests
print(list(fibonacci(1)))  # [0]
print(list(fibonacci(5)))  # [0, 1, 1, 2, 3]


[0]
[0, 1, 1, 2, 3]


## Exo 4 — Générateur infini `count_from(start=0)`
- Générateur **infini** qui produit `start, start+1, start+2, ...`
- Test : appelez `next()` plusieurs fois. Attention à ne pas convertir en `list(...)` !


In [4]:
# TODO: générateur infini
def count_from(start: int = 0):
    # TODO: boucle infinie, yield, puis incrément
    cur = int(start)
    while True:
        yield cur
        cur += 1

# tests (ne pas lister !)
it = count_from(10)
print(next(it))  # 10
print(next(it))  # 11
print(next(it))  # 12


10
11
12


## Exo 5 — Générateur `read_lines(path)`
- Lire un fichier **ligne par ligne** en mode lazy (ne pas charger tout en mémoire).
- Utiliser `with open(..., encoding="utf-8")`.
- `yield line.strip()` pour renvoyer lignes sans fin de ligne.

*(si vous n’avez pas de fichier, créez-en un petit avec 3–4 lignes)*


In [5]:
from pathlib import Path

# TODO: read_lines(path)
def read_lines(path: Path):
    # TODO: ouvrir et yield chaque ligne strip()
    with open(path, encoding="utf-8") as f:
        for line in f:
            yield line.strip()

# préparation test (dé commentez si besoin)
Path("demo.txt").write_text("alpha\nbeta\ngamma\n", encoding="utf-8")
for line in read_lines(Path("demo.txt")):
    print(line)  # alpha / beta / gamma


alpha
beta
gamma


## Exo 6 — Pipeline de générateurs
- `numbers()` : yield de 1 à 100
- `squares(it)` : yield le carré de chaque nombre
- `even_filter(it)` : ne garder que les valeurs **pairs**

Test :
- `list(even_filter(squares(numbers())))`


In [6]:
# TODO: numbers, squares, even_filter
def numbers():
    # TODO: yield 1..100
    for n in range(1, 101):
        yield n

def squares(it):
    # TODO: yield n*n pour n dans it
    for n in it:
        yield n * n

def even_filter(it):
    # TODO: yield n seulement si n % 2 == 0
    for n in it:
        if n % 2 == 0:
            yield n

# test
# print(list(even_filter(squares(numbers()))))

res = list(even_filter(squares(numbers())))
print(res[:10], "...", len(res))  # premiers 10 et taille totale


[4, 16, 36, 64, 100, 144, 196, 256, 324, 400] ... 50


## Exo 7 — `chunked(iterable, size)`
- Écrire un générateur qui découpe un iterable en **blocs** de taille `size` (dernier bloc plus petit possible).
- Exemple : `list(chunked([1,2,3,4,5,6,7], 3))` → `[[1,2,3],[4,5,6],[7]]`.
- Contrainte : ne pas tout charger si l'iterable est gros → itérer et accumuler par paquets.


In [7]:
# TODO: chunked(iterable, size)
def chunked(iterable, size: int):
    # TODO: accumuler dans un buffer jusqu'à size, yield, puis vider
    buf = []
    for item in iterable:
        buf.append(item)
        if len(buf) == size:
            yield buf
            buf = []
    if buf:
        yield buf

# tests
print(list(chunked([1,2,3,4,5,6,7], 3)))  # [[1,2,3],[4,5,6],[7]]


[[1, 2, 3], [4, 5, 6], [7]]


## Exo 8 — `unique(iterable)` (ordre d'apparition conservé)
- Yield uniquement la **première occurrence** de chaque valeur, en conservant l’ordre d’arrivée.
- Exemple : `list(unique([1,2,2,3,1,4]))` → `[1,2,3,4]`.
- Conseil : utiliser un `set` pour mémoriser les vues.


In [8]:
# TODO: unique(iterable)
def unique(iterable):
    # TODO: utiliser un set 'seen' pour filtrer
    seen = set()
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item

# tests
print(list(unique([1,2,2,3,1,4])))   # [1,2,3,4]
print("".join(unique("banane")))     # "bane"


[1, 2, 3, 4]
bane


## ✅ Checklist J5
- [X] Itérateur de base (iter/next)
- [X] Itérateur custom (EvenNumbers)
- [X] Générateur fini (fibonacci)
- [X] Générateur infini (count_from)
- [X] Générateur lazy fichier (read_lines)
- [X] Pipeline de générateurs
- [X] chunked(iterable, size)
- [X] unique(iterable)

### Suite (prépa Flask)
- Réutiliser `read_lines` et `unique` pour traiter des données simples (ex: log, CSV).
- Garder `chunked` en tête pour la pagination côté serveur.
